In [7]:
import camelot
import pandas as pd
import altair as alt
from bs4 import BeautifulSoup
import requests
import re

In [8]:

def filter_counties(df):
    in_massachusetts = df["state"] == "Massachusetts"
    in_mwra_counties = df["county"].isin(["Middlesex", "Suffolk", "Norfolk", "Essex"])
    return df[in_massachusetts & in_mwra_counties]

In [9]:
# load mwra data, finding the url for the latest data pdf
mwra_home = "https://www.mwra.com/biobot/"
with requests.get(mwra_home + "biobotdata.htm") as page:
    soup = BeautifulSoup(page.content)
    view_the_data = soup.find(string=re.compile(r'view\s+the\s+data', re.I))
    url = mwra_home + view_the_data.nextSibling['href']
    pdf = camelot.read_pdf(url, pages="all")

In [10]:
# load nyt data
county_data_2022 = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/rolling-averages/us-counties-2022.csv"
county_data_2021 = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/rolling-averages/us-counties-2021.csv"
county_data_2020 = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/rolling-averages/us-counties-2020.csv"
county_data = pd.concat((
    filter_counties(pd.read_csv(county_data_2022, parse_dates=["date"])),
    filter_counties(pd.read_csv(county_data_2021, parse_dates=["date"])),
    filter_counties(pd.read_csv(county_data_2020, parse_dates=["date"])),
))
county_data = county_data.drop(columns=["geoid", "state"])

ValueError: Missing column provided to 'parse_dates': 'date'

In [ ]:
# combine all pages
df = pd.concat([page.df.iloc[:, :9] for page in pdf])

# set header row
df.columns = df.iloc[0]
df = df.drop(0)
new_column_names = {name: name.replace("\n", "") for name in df.columns}
df = df.rename(columns=new_column_names)

# convert columns to proper types
df = df.rename(columns={"Sample Date": "date"})
df["date"] = pd.to_datetime(df["date"], errors="coerce")
for col in df.columns:
    if col != "date":
        df[col] = pd.to_numeric(df[col], errors="coerce")

# drop all rows with only the date value
df = df.dropna(thresh=2) 

# extract region (Southern / Northern) from column headers
copies_ml = df.melt(id_vars=["date"], value_vars=("Southern (copies/mL)", "Northern (copies/mL)"), var_name="county", value_name="copies_per_ml")
copies_ml["county"] = copies_ml["county"].map({"Southern (copies/mL)": "Southern", "Northern (copies/mL)": "Northern"})
copies_ml_avg = df.melt(id_vars=["date"], value_vars=("Southern 7 day avg", "Northern 7 day avg"), var_name="county", value_name="copies_per_ml_avg")
copies_ml_avg["county"] = copies_ml_avg["county"].map({"Southern 7 day avg": "Southern", "Northern 7 day avg": "Northern"})

confidence_interval_high = df.melt(id_vars=["date"], value_vars=("Southern High Confidence Interval", "Northern High Confidence Interval"), var_name="county", value_name="confidence_interval_high")
confidence_interval_high["county"] = confidence_interval_high["county"].map({"Southern High Confidence Interval": "Southern", "Northern High Confidence Interval": "Northern"})
confidence_interval_low = df.melt(id_vars=["date"], value_vars=("Southern Low Confidence Interval", "Northern Low Confidence Interval"), var_name="county", value_name="confidence_interval_low")
confidence_interval_low["county"] = confidence_interval_low["county"].map({"Southern Low Confidence Interval": "Southern", "Northern Low Confidence Interval": "Northern"})

dfs = [copies_ml, copies_ml_avg, confidence_interval_high, confidence_interval_low]
df = pd.merge(copies_ml, copies_ml_avg, on=["date", "county"])
df = pd.merge(df, confidence_interval_high, on=["date", "county"])
df = pd.merge(df, confidence_interval_low, on=["date", "county"])

# combine NYT data and MWRA data
combined = pd.merge(county_data, df, on=["date", "county"], how="outer")
combined = combined.rename(columns={"county": "region"})
combined.to_csv("covid-19-wastewater.csv", index=False)

In [ ]:
def plot(df, nyt_domain):
    # MWRA wastewater covid measurement
    mwra_color = "#1b9e77"
    mwra_data = df[df["region"] == "Northern"]
    mwra_data["min_error"] = mwra_data["copies_per_ml_avg"] - mwra_data["confidence_interval_low"]
    mwra_data["max_error"] = mwra_data["copies_per_ml_avg"] + mwra_data["confidence_interval_high"]
    mwra_max = mwra_data["copies_per_ml_avg"].max()
    mwra_avg = alt.Chart(mwra_data).mark_line(color=mwra_color, size=1).encode(
        alt.X("date", title=None, axis=alt.Axis(format="%b %Y", grid=False)),
        alt.Y(
            "copies_per_ml_avg",
            title="RNA copies per mL of wastewater, weekly avg (Northern MWRA region)",
            scale=alt.Scale(domain=(0, mwra_max * 1.1)),
            axis=alt.Axis(
                domainColor=mwra_color,
                tickColor=mwra_color,
                labelColor=mwra_color,
                titleColor=mwra_color,
                domain=False,
                values=[0, mwra_max],
                titlePadding=20,
            ),
        )
    )
    mwra_errors = alt.Chart(mwra_data).mark_area(color=mwra_color, opacity=0.2).encode(
        alt.X("date", title=None, axis=alt.Axis(format="%b %Y", grid=False)),
        y="min_error",
        y2="max_error",
    )

    # New York Times covid cases
    nyt_color = "#d95f02"
    nyt_data = df[df["region"] == "Middlesex"]
    nyt_max = nyt_data["cases_avg_per_100k"].max()
    nyt_avg = alt.Chart(nyt_data).mark_line(color=nyt_color, size=1.5).encode(
        alt.X("date", title=None, axis=alt.Axis(format="%b %Y", grid=False)),
        alt.Y(
            "cases_avg_per_100k",
            title="Cases per 100k, weekly avg (Middlesex, MA)",
            scale=alt.Scale(domain=nyt_domain),
            axis=alt.Axis(
                domainColor=nyt_color,
                tickColor=nyt_color,
                labelColor=nyt_color,
                titleColor=nyt_color,
                values=[0, nyt_max],
                gridColor=nyt_color,
                domain=False,
                titleAlign="left",
            ),
        )
    )

    # New York Times covid deaths
    deaths_color = "#444"
    deaths_max = nyt_data["deaths_avg_per_100k"].max()
    nyt_deaths = alt.Chart(nyt_data).mark_area(color=deaths_color, opacity=0.1).encode(
        alt.X("date", title=None, axis=alt.Axis(format="%b %Y", grid=False)),
        alt.Y(
            "deaths_avg_per_100k",
            title="Deaths per 100k, weekly avg (Middlesex, MA)",
            scale=alt.Scale(domain=[0, 5]),
            axis=alt.Axis(
                domainColor=deaths_color,
                tickColor=deaths_color,
                labelColor=deaths_color,
                titleColor=deaths_color,
                values=[0, deaths_max],
                gridColor=deaths_color,
                domain=False,
                gridOpacity=0.3,
                grid=True,
            ),
        ),
    )

    # Combine them on the same plot
    return alt.layer(nyt_deaths, mwra_errors + mwra_avg, nyt_avg).resolve_scale(
        y = 'independent'
    ).properties(
        width=500,
        height=500,
    ).configure_axisX(
        labelAngle=-90,
    ).configure_axisY(
        titleFontSize=15,
        titleFontWeight="normal",
    ).configure_view(
        strokeWidth=0,
    )

visualization = plot(combined, (0, 500))
visualization.save("covid-19-wastewater.json")
visualization

alt.LayerChart(...)